In [1]:
import pandas as pd
import re
import numpy as np
from tqdm import tqdm

In [2]:
df = pd.read_csv("./Train/train.csv")

In [3]:
df.head()

,Id,Title,Body,Tags
0,1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...
1,2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox
2,3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning
3,4,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding
4,5,How to modify whois contact details?,<pre><code>function modify(.......)\n{\n $mco...,php api file-get-contents


In [4]:
df.shape

(6034195, 4)

In [5]:
# Remove all the data points that exactly match the field values mention below

data = df.drop_duplicates(subset={'Id', 'Title', 'Body', 'Tags'}, keep='first', inplace=False)
data.shape

(6034195, 4)

In [6]:
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [7]:
# This cell need not run every time but only once at the beginning

import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\MSI
[nltk_data]     Gaming\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# Printing the set of stop words present in english language
# You need to download stopword first before using it(import nltk; nltk.download("stopwords");)

stop_words = set(stopwords.words('english'))
print(stop_words)

{'this', 'themselves', 'those', 'down', 'she', 'after', 've', 'while', 'once', 'your', "hasn't", 'no', 'further', "needn't", 'who', 'other', 'few', 'should', 'wouldn', 'any', 'when', 'not', 'they', 'mustn', 'do', 'very', 'that', 'it', 'from', "you're", "doesn't", 'during', 'which', 'some', 'an', 'can', 'her', 's', 'about', "couldn't", "it's", 'had', 'before', 'its', 'll', 'wasn', 'my', 'same', 'me', 'between', 'are', 'where', "haven't", 'own', 'there', 'shan', "that'll", "should've", 'their', 'at', "didn't", 'as', 'itself', 'ma', 'below', 'off', 'why', 'than', 'on', 'myself', 'and', "won't", 'again', 't', 'has', 'if', 'd', 'the', 'such', "mightn't", 'until', "shouldn't", 'each', 'doesn', 'through', 'hers', 'for', 'am', "hadn't", 'shouldn', 'or', 'against', 'were', 'more', 'being', 'out', 'but', 'hadn', 'haven', 'm', 'now', 'needn', 'weren', 'herself', 'up', 'theirs', 'him', 'above', 'both', 'because', 'by', 'have', 'yourselves', 'only', 'them', "shan't", 'to', 'don', "weren't", 'mightn

In [9]:
# Initializing SnowBall Stemmer
# Stemmer convert the given word to its root word form

snowball = SnowballStemmer('english')

In [10]:
# Function to remove texts with html tags(<...>)

def clean_html(sentence):
    # Select ANY text with < > tags
    selected_text = re.compile('<.*?>')
    # Replace texts enclosed in < > with white space
    cleaned_text = re.sub(selected_text, ' ', sentence)
    return cleaned_text

In [11]:
# Function to remove punctuation marks(.,?/:;)

def clean_punctuation(sentence):
    cleaned_text = re.sub(r'[?|$|.|!]',r'', sentence)
    cleaned_text = re.sub(r'[.|,|)|(|\|/]',r' ', cleaned_text)
    return cleaned_text

In [12]:
# Core Pre-processing code step-by-step(removing html tags, punctuations, stopwords, etc)
# This may take a few minutes to execute as it needs to execute all the sentences present in data

def preprocess_text(column_name):
    initial_str = ''
    final_str = []          # Store final string of clean words
    s = ''

    for sentence in tqdm(data[column_name].values):
        filtered_sentence = []
        sentence = clean_html(sentence)          # Remove html tags
        for words in sentence.split():
            for word in clean_punctuation(words).split():
                if ((word.isalpha()) & (len(word) > 2)):
                    if (word.lower() not in stop_words):
                        s = (snowball.stem(word.lower()).encode('utf-8'))
                        filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue
        initial_str = b" ".join(filtered_sentence)      # Final string of clean words
        final_str.append(initial_str)
    return final_str

In [13]:
data_title = preprocess_text("Title")

# Adding a column in dataset and storing cleaned text in it.
data['cleaned_title'] = data_title
data['cleaned_title'] = data['cleaned_title'].str.decode("utf-8")

  7%|█████▎                                                                 | 448419/6034195 [02:19<27:09, 3428.83it/s]

KeyboardInterrupt: 

In [ ]:
data.head()

In [ ]:
data_body = preprocess_text("Body")

# Adding a column in dataset and storing cleaned text in it.
data['cleaned_body'] = data_body
data['cleaned_body'] = data['cleaned_body'].str.decode("utf-8")

In [ ]:
data.head()

In [ ]:
data.to_csv("./cleaned_text.csv", index=False)